In [1]:
import altair as alt
import pandas as pd
import glob
import os

In [2]:
topics = pd.read_csv('../results/bert_probabilities_merged_85_new_names.csv', sep=',')
topic_names = topics.set_index('Topic')['CustomName']
csv_files = glob.glob('../results/discourse_matrices/*.csv')

In [3]:
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    df.drop(columns=['Unnamed: 0'], inplace=True)
    df = df * 100
    df['sor'] = df.index
    value_vars = df.columns[:-1]  # Exclude the 'sor' column
    df_melted = pd.melt(
        df,
        id_vars=['sor'],
        value_vars=value_vars
    )
    df_melted['topik'] = df_melted['variable']
    df_melted.drop(columns=['variable'], inplace=True)
    df_melted['topik_nev'] = df_melted['topik'].apply(lambda x: topic_names[int(x)])
    alt.themes.enable('fivethirtyeight')
    chart = (
        alt.Chart(df_melted)
        .mark_circle()
        .encode(
            x=alt.X('sor:N', axis=alt.Axis(labelAngle=-45)),
            y='topik:N',
            size=alt.Size('value:Q', scale=alt.Scale(range=[0, 1000]), title=None, legend=None),
            color=alt.Color(
                'topik_nev:N',
                scale=alt.Scale(
                    range=[
                        'blue', 'red', 'green', 'yellow', 'purple', 'orange', 'brown', 'black', 'grey', 'violet', 'cyan', 'magenta', 'olive', 'maroon', 'lime', 'teal', 'indigo', 'violet', 'pink', 'gold', 'silver', 'hotpink', 'midnightblue', 'darkorchid', 'rosybrown'
                    ]
                ),
                legend=alt.Legend(title='Topik')
            ),
            tooltip=['topik_nev', 'value'],
        )
    ).interactive()
    output_file = os.path.join('../vizs/punchcard_per_discourse', f'punchcard_{os.path.basename(csv_file).split(".")[0]}.html')
    chart.save(output_file)